In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import os
import json
import pprint
%matplotlib inline
import tweepy
import networkx as nx
from pyvis.network import Network
import pprint
import random
print("imported completed!")

imported completed


In [ ]:
consumer_key = "GvDHFIS3fpCfZR2hwEGhE5oCZ"
consumer_secret = "LCwgUZgFOjJppd4jg6Cf4mgohUTc6Ro7Yaals87LdNLmS3d9zB"
access_token = "810960428385890304-aVlOFR9EExKdDPZCsh5EDN5Nm9SUmb9"
access_secret = "XJkr9dKABPkSvRIgpyQeNapg2o6ldP4VoNbTids9QXGWT"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAA8UJQEAAAAAOD26yUUUPHF9zeAe56TtzGYUK8Q%3DRh6u9TAI7T7b5TJABTBOP5WwWSL0Cbvv5a6JJwg8MursyOOt9L"

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)
if api.verify_credentials :
  print ('Authentication completed successfully!')

Authentication completed successfully!


In [ ]:
data_folder =  "data"
def serialize_json    (folder, filename, data):
  if not os.path.exists(folder):
    os.makedirs(folder, exist_ok=True)
  with open(f"{folder}/{filename}", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
  print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
  if os.path.exists(path):
    with open(path, "r", encoding="utf8") as file:
      data = json.load(file)
    print(f"Data read fron path: {path}")
    return data
  else:
    print(f"No data found at path: {path}")
    return {}

def usersID (users):
    usersID = []
    for user in users:
        json_data = api.get_user(user)._json
        usersID.append(json_data["id"])
    return usersID

def download_followers(idUser, quantity):
  followers_of_user = []
  for follower in tweepy.Cursor(
        api.followers,
        id = user,
        skip_status=True,
        include_user_entities = False
    ).items(quantity):
        #json_data = follower._json
        followers_of_user.append(follower._json["id"])
  return followers_of_user

def download_following(user, quantity):
  friends_of_user = []
  for friend in tweepy.Cursor(
        api.friends,
        id = user,
        skip_status=True,
        include_user_entities = False
    ).items(quantity):
        json_data = friend._json
        friends_of_user.append(friend._json["id"])
  return friends_of_user

In [ ]:
users = ["lucabazzetto","Tomve99"]
#usersID = usersID(users)
quantity = 800
followers = {}
for user in users:
    followers_of_user = []
    for follow in tweepy.Cursor(
        api.followers,
        screen_name = user,
        skip_status=True,
        include_user_entities = False
    ).items(quantity):
        json_data = follow._json
        if json_data not in followers_of_user:
            followers_of_user.append(json_data["screen_name"])
    followers[user] = followers_of_user
serialize_json(data_folder,"followersLucaThomas.json", followers)

Data serialized to path: data/followersLucaThomas.json


In [ ]:
users = ["lucabazzetto","Tomve99"]
quantity = 800
friends = {}
for user in users:
    friends_of_user = []
    for friend in tweepy.Cursor(
        api.friends,
        screen_name = user,
        skip_status=True,
        include_user_entities = False
    ).items(quantity):
        json_data = friend._json
        friends_of_user.append(friend._json["screen_name"])
    friends[user] = friends_of_user
serialize_json(data_folder,"friendsLucaThomas.json", friends)

Data serialized to path: data/friendsLucaThomas.json


In [ ]:
def random_json_with_control(json_array, quantity, all):
    a = quantity
    array = []
    while(a!=0):
        item = random.choice(json_array)
        if item not in array and item not in all:
            array.append(item)
            a = a-1
    return array

followersALL = read_json(f"{data_folder}/followersLucaThomas.json")
fiveFollowers = []

# Estraggo per ogni utente 5 followers
for user in followersALL:
    fiveFollowers.extend(random_json_with_control(followersALL[user], 3, fiveFollowers))

quantity = 10
followersOfFollowers = {}

for user in fiveFollowers:
    followers_of_user = []
    for follower in tweepy.Cursor(
        api.followers,
        screen_name = user,
        skip_status=True,
        include_user_entities = False
    ).items(quantity):
        json_data = follower._json
        followers_of_user.append(follower._json["screen_name"])
    followersOfFollowers[user] = followers_of_user

serialize_json(data_folder,"followersOfFollowers.json", followersOfFollowers)

Data read fron path: data/followersLucaThomas.json
Data serialized to path: data/followersOfFollowers.json


In [ ]:
friendsALL = read_json(f"{data_folder}/friendsLucaThomas.json")
fiveFriends = []

# Estraggo per ogni utente 5 followers
for user in friendsALL:
    fiveFriends.extend(random_json_with_control(friendsALL[user], 5, fiveFriends))

quantity = 10
friendsOfFriends = {}
for user in fiveFriends:
    friends_of_user = []
    for friend in tweepy.Cursor(
        api.friends,
        screen_name = user,
        skip_status=True,
        include_user_entities = False
    ).items(quantity):
        json_data = friend._json
        friends_of_user.append(friend._json["screen_name"])
    friendsOfFriends[user] = friends_of_user
serialize_json(data_folder,"friendsOfFriends.json", friendsOfFriends)

Data read fron path: data/friendsLucaThomas.json
Data serialized to path: data/friendsOfFriends.json


In [ ]:
allFriends = {}
allFriends.update(read_json(f"{data_folder}/friendsOfFriends.json"))
allFriends.update(read_json(f"{data_folder}/followersOfFollowers.json"))
allFriends.update(read_json(f"{data_folder}/followersLucaThomas.json"))
allFriends.update(read_json(f"{data_folder}/friendsLucaThomas.json"))
#serialize_json(data_folder,"allFriends.json", allFriends)
#pprint.PrettyPrinter().pprint(allFriends)

vector = []
for (k,v) in allFriends.items():
    for item in v:
        if item not in vector:
            vector.append(item)

serialize_json(data_folder,"NODUPLICATI.json", vector)

puntoQuattro = {}

for user in vector:
    json_data = api.get_user(user)._json
    foundInfo = {}
    foundInfo ["id"] = json_data["id"]
    foundInfo ["name"] = json_data["name"]
    foundInfo ["screen_name"] = json_data["screen_name"]
    foundInfo ["location"] = json_data["location"]
    foundInfo ["followers_count"] = json_data["followers_count"]
    foundInfo ["friends_count"] = json_data["friends_count"]
    puntoQuattro[user] = foundInfo
serialize_json(data_folder,"puntoQuattro.json", puntoQuattro)


Data read fron path: data/friendsOfFriends.json
Data read fron path: data/followersOfFollowers.json
Data read fron path: data/followersLucaThomas.json
Data read fron path: data/friendsLucaThomas.json
Data serialized to path: data/NODUPLICATI.json
Data serialized to path: data/puntoQuattro.json


In [ ]:
allDataPoint4 = read_json(f"{data_folder}/puntoQuattro.json")
graph = nx.DiGraph()
graph.add_nodes_from(allDataPoint4)

graph.graph["members"] = ["Luca","Thomas","Agustin","Edoardo"]

for user in allDataPoint4:
    for user2 in allDataPoint4:
        friendship = api.show_friendship(source_screen_name = user,target_screen_name = user2)
        if friendship[0].followed_by:
            graph.add_edge(user2,user)
        if friendship[0].following:
            graph.add_edge(user,user2)

nx.write_gpickle(graph,"data/graph.pkl")

Data read fron path: data/puntoQuattro.json


Rate limit reached. Sleeping for: 866


In [ ]:

nt = Network(
    height="100%",
    width="100%",
    bgcolor="222222",
    font_color="white",
    heading="progetto"
)
nt.barnes_hut()
nt.from_nx(graph)
neighbor_map = nt.get_adj_list()
for node in nt.nodes:
    node["Value"] = len(neighbor_map[node['id']])
nt.show("grafico.html")

In [ ]:
username = "lucabazzetto"
quantity = 10
AllUsers = {}
dataOfUser = []
for follower in tweepy.Cursor(
    api.followers,
    screen_name = username,
    skip_status=True,
    include_user_entities = False
).items(quantity):
    dataOfUser.append(follower._json["screen_name"])

for friend in tweepy.Cursor(
    api.friends,
    screen_name = username,
    skip_status=True,
    include_user_entities = False
).items(quantity):
    jsonData = friend._json["screen_name"]
    if jsonData not in dataOfUser:
        dataOfUser.append(jsonData)

AllUsers[username] = dataOfUser
serialize_json(data_folder,"lucabazzettoALL.json", AllUsers)

Data serialized to path: data/lucabazzettoALL.json


In [ ]:
allData = read_json(f"{data_folder}/lucabazzettoALL.json")
pp = pprint.PrettyPrinter()
pp.pprint(allData)

Data read fron path: data/lucabazzettoALL.json
{'lucabazzetto': ['AndroidGamerIT',
                  'giucruciani',
                  'aledellagiusta',
                  'MKBHD',
                  'getpeid',
                  'OnePlus_ITA',
                  'elonmusk',
                  'berlusconi',
                  'CRAZYFATGAMER',
                  'EAFIFADirect',
                  'NicolaPorro']}


In [ ]:
details = api.get_user("lucabazzetto")._json
serialize_json(data_folder,"details.json", details)
pp.pprint(details)

Data serialized to path: data/details.json
{'contributors_enabled': False,
 'created_at': 'Mon Dec 19 21:30:07 +0000 2016',
 'default_profile': True,
 'default_profile_image': True,
 'description': '',
 'entities': {'description': {'urls': []}},
 'favourites_count': 17,
 'follow_request_sent': False,
 'followers_count': 1,
 'following': False,
 'friends_count': 27,
 'geo_enabled': False,
 'has_extended_profile': True,
 'id': 810960428385890304,
 'id_str': '810960428385890304',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': None,
 'listed_count': 0,
 'location': '',
 'name': 'Luca Bazzetto',
 'needs_phone_verification': False,
 'notifications': False,
 'profile_background_color': 'F5F8FA',
 'profile_background_image_url': None,
 'profile_background_image_url_https': None,
 'profile_background_tile': False,
 'profile_image_url': 'http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png',
 'profile_image_url_https': 'https://abs.twimg.com/sticky/d